In [35]:
import sys
paths_to_add = ["/homes/vk223/ProjectFlood", "/home/vkhandekar/project_flood"]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)
print(sys.path)
# import ipyleaflet
import geopandas as gpd
from shapely.geometry import box
import os
print(os.getcwd())
# os.chdir("/home/vkhandekar/project_flood")
# import rioxarray
# from pathlib import Path
import ee
# import geemap.core as geemap
import geemap
import xarray as xr
import cdsapi
import zipfile
import json
import pandas as pd
import numpy as np
import rasterio
import uuid
from rasterio.mask import mask
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.geometry import Polygon, mapping, box
import matplotlib.pyplot as plt
# from osgeo import gdal
from scipy.ndimage import zoom
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# import cfgrib
print(os.environ['PROJECT_FLOOD_DATA'])


['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/vkhandekar/project_flood/.venv/lib/python3.10/site-packages', '/homes/vk223/ProjectFlood', '/home/vkhandekar/project_flood']
/home/vkhandekar/project_flood/data_extraction
/home/vkhandekar/project_flood/static/local_data_paths.json


In [3]:
ee.Authenticate()
ee.Initialize(project='ee-varunkhandekar')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [32]:
shapefile_path = '/home/vkhandekar/project_flood/static/bangladesh-outline_68.geojson'
# 'bangladesh-detailed-boundary_858.geojson' 'bangladesh-outline_68.geojson'


with open(shapefile_path, 'r') as file:
    geojson_data = json.load(file)

# Extract polygon coordinates
polygon_coordinates = []

for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'Polygon':
        polygon_coordinates.append(feature['geometry']['coordinates'])
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            polygon_coordinates.append(polygon)

ee_geometry = ee.Geometry.MultiPolygon(polygon_coordinates)
# Create the polygon
polygon = Polygon(polygon_coordinates[0][0])

# print(ee_geometry.getInfo()['coordinates'])
# print(polygon_coordinates[0])
# print(ee_geometry.getInfo()['coordinates']==polygon_coordinates[0])
print(polygon.bounds)
print(ee_geometry.bounds().getInfo())
bbox_ee = ee.Geometry.BBox(*polygon.bounds)

bbox = box(*polygon.bounds)

geojson = [mapping(bbox)]
geojson


def generate_square_coordinates_from_polygon(polygon: Polygon):
    min_lon, min_lat, max_lon, max_lat = polygon.bounds
    width = max_lon - min_lon
    height = max_lat - min_lat
    square_size = max(width, height)

    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    new_min_lon = center_lon - square_size / 2
    new_min_lat = center_lat - square_size / 2
    new_max_lon = center_lon + square_size / 2
    new_max_lat = center_lat + square_size / 2
    return new_min_lon, new_min_lat, new_max_lon, new_max_lat



bangladesh_bounding_box_square_ee = ee.Geometry.BBox(*generate_square_coordinates_from_polygon(polygon))

(88.084422, 20.670883, 92.672721, 26.446526)
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[88.084422, 20.670882999999975], [92.672721, 20.670882999999975], [92.672721, 26.446526000000024], [88.084422, 26.446526000000024], [88.084422, 20.670882999999975]]]}


In [ ]:
# precipitation_upsampled
# ERA5-LAND
# BangladeshTopology
# BangladeshFloodedCombo_MASTER
# BangladeshFloodedCombo_vF
with rasterio.open("precipitation_upsampled.tif") as src:
    dataset = src.read()

print(dataset)
print(dataset.dtype)
print(dataset.max())
print(dataset.mean())
print(dataset.min())
print(dataset.size)
print(dataset.shape)

Initial Exploration with Landsat Dataset

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')


point = ee.Geometry.Point(-122.1899, 37.5010)

# Get the pixel values at the defined point
pixel_values = image.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
)

# Print the pixel values to the console
print(pixel_values.getInfo())


In [ ]:

# Extract individual bands
band5 = image.select('B5')
band4 = image.select('B4')
band3 = image.select('B3')

# Apply a color palette to the bands
band5_colored = band5.visualize(min=0, max=0.5, palette=['#add8e6'])  # Light blue
band4_colored = band4.visualize(min=0, max=0.5, palette=['#00008b'])  # Dark blue
band3_colored = band3.visualize(min=0, max=0.5, palette=['#008b8b'])  # Cyan as an example

# Combine the colored bands into an RGB image
rgb_image = ee.ImageCollection([band5_colored, band4_colored, band3_colored]).mosaic()

# Define a map centered on San Francisco Bay.
map_l8 = geemap.Map(center=[37.5010, -122.1899], zoom=10)

# Add the image layer to the map and display it.
map_l8.addLayer(rgb_image, {}, 'Custom Color Composite')
map_l8

In [ ]:
Map = geemap.Map()

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(
    ['B5', 'B4', 'B3']
)

vis_params = {'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252) #bottom left to top right
clipped_image = image.clip(region) #clipping event to bounding box


Map.centerObject(image, 8)
Map.addLayer(image, vis_params, 'Landsat')
# Map.addLayer(clipped_image, vis_params, 'Landsat')
Map

In [ ]:

fc = ee.FeatureCollection(region)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')
Map

In [ ]:
geemap.ee_export_image(image, filename="landsat.tif", scale=30, region=region)

In [ ]:
# # Load the shapefile
# shapefile_path = 'hybas_as_lev05_v1c.shp'
# gdf = gpd.read_file(shapefile_path)

# # Define the bounding box (xmin, ymin, xmax, ymax)
# bbox = box(xmin, ymin, xmax, ymax)

# # Create a GeoDataFrame for the bounding box
# bbox_gdf = gpd.GeoDataFrame([[1]], geometry=[bbox], crs=gdf.crs)

# # Find the polygons that intersect with the bounding box
# intersecting_polygons = gdf[gdf.intersects(bbox)]

# # Output the result
# print(intersecting_polygons)

# # Optionally, save the intersecting polygons to a new shapefile
# intersecting_polygons.to_file('intersecting_polygons.shp')



Permanent Water Overlay

In [ ]:
Map = geemap.Map()

# Load the dataset and filter for the month of December
dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
im = ee.Image(dataset.filterMetadata('month', 'equals', 12).first())
print(im.getInfo())

occurrence_band = im.select('monthly_recurrence')

# Apply a threshold to filter values greater than x%
filtered_occurrence = occurrence_band.gt(25) #boolean output
print(filtered_occurrence.getInfo())

point = ee.Geometry.Point(89.97, 23.65)

# Get the pixel values at the defined point
pixel_values = filtered_occurrence.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
)
# Print the pixel values to the console
print(pixel_values.getInfo())

# Set visualization parameters
visualization = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['blue']
}

# Add the filtered layer to the map
Map.setCenter(89.42, 24.23, 9)
Map.addLayer(filtered_occurrence.selfMask(), visualization, 'Monthly Recurrence')


fc = ee.FeatureCollection(ee_geometry)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')

# Display the map
Map

Topology

In [ ]:
# "NASA/NASADEM_HGT/001"

Map = geemap.Map()

# Load the dataset and filter for the month of December
# dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
bangladeshTopology = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
print(bangladeshTopology.getInfo())

# Set visualization parameters
visualization = {
    'min': 0.0,
    'max': 2000.0,
}

# Replace all no data values with 0
bangladeshTopology = bangladeshTopology.unmask(0)

# kernel = ee.Kernel.gaussian(radius=1, sigma=1, units='pixels', normalize=True)
# smoothed_im = im.convolve(kernel)

# background = ee.Image(1);
# Map.addLayer(background, {min: 0, max: 1});
# print(background.getInfo())

# Add the filtered layer to the map
Map.setCenter(89.42, 24.23, 9)
# Map.addLayer(smoothed_im, visualization, 'Elevation')
Map.addLayer(bangladeshTopology, visualization, 'Elevation')


print(ee_geometry.getInfo())

fc = ee.FeatureCollection(ee_geometry)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')

# Display the map
Map

In [37]:
# id = str(uuid.uuid4())
export_task = ee.batch.Export.image.toDrive(
    image=bangladeshTopology,
    description='BangladeshTopology',
    folder='Topology',
    fileNamePrefix='uuidtest',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=[0.0226492347869873, 0.0, 88.08430518433968, 0.0, -0.0226492347869873, 26.44864775268901],
    maxPixels=1e13  # Adjust maxPixels as needed
    # requestId=id
)
export_task.start()

EEException: Unknown configuration options: {'requestId': 'aaeb8d44-6164-456c-839c-87a8387ee6d5'}.

In [ ]:
bangladeshTopology = bangladeshTopology.resample(mode="bilinear")
export_task = ee.batch.Export.image.toDrive(
    image=bangladeshTopology,
    description='BangladeshTopology_bilinear',
    folder='Topology',
    fileNamePrefix='BangladeshTopology_bilinear',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=[0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901],
    maxPixels=1e13  # Adjust maxPixels as needed
)
export_task.start()

In [ ]:
export_task.status()

In [ ]:
with rasterio.open('BangladeshTopology_bilinear.tif') as src:
    # Read the existing data
    data = src.read()
    
data.max()

In [ ]:
with rasterio.open('BangladeshTopology.tif') as src:
    # Read the existing data
    data = src.read()
    
data.max()

Mosaicing Overlapping Flood Events

In [ ]:
Map = geemap.Map()
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

bangladeshID1 = 1789
bangladeshID2 = 1791
bangladesh1 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID1).first()).select(['flooded'])
bangladesh2 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID2).first()).select(['flooded'])

# combo = ee.ImageCollection([bangladesh1, bangladesh2]).reduce(ee.Reducer.max())
# vis_params = {'min': 0, 'max': 1, 'palette': 'C3EFFE'}

vis_params1 = {'min': 0, 'max': 1, 'palette': '#00FFFF'}
Map.centerObject(bangladesh1, 9)
Map.addLayer(bangladesh1.selfMask(), vis_params1, '1 - Inundation Extent')
vis_params2 = {'min': 0, 'max': 1, 'palette': '#FF0000'}
Map.centerObject(bangladesh2, 9)
Map.addLayer(bangladesh2.selfMask(), vis_params2, '2 - Inundation Extent')
Map

In [4]:
Map = geemap.Map()
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

# Load the dataset
dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
im = ee.Image(dataset.filterMetadata('month', 'equals', 12).first())

occurrence_band = im.select('monthly_recurrence')

# Apply a threshold to filter values greater than x%
filtered_occurrence = occurrence_band.gt(50)
filtered_occurrence = filtered_occurrence.rename('flooded')
# filtered_occurrence = filtered_occurrence.resample(mode="bilinear")

# print(filtered_occurrence.projection().crs().getInfo())

filtered_occurrence = filtered_occurrence.setDefaultProjection("EPSG:4326", [0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901])
filtered_occurrence = filtered_occurrence.reduceResolution(reducer=ee.Reducer.mode())
print(filtered_occurrence.getInfo())

sea = ee.Image(ee.ImageCollection('MODIS/006/MOD44W').filterDate('2015-01-01', '2015-01-02').first()).select(['water_mask'])
sea = sea.gt(0.5)
sea = sea.rename('flooded')
sea = sea.setDefaultProjection("EPSG:4326", [0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901])
# sea = sea.resample(mode="bicubic")
print(sea.getInfo())

bangladeshID1 = 1789
bangladeshID2 = 1791
bangladesh1 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID1).first()).select(['flooded'])
bangladesh1 = bangladesh1.gt(0.5)
# bangladesh1 = bangladesh1.resample(mode="bilinear")
# bangladesh1 = bangladesh1.reduceResolution(reducer=ee.Reducer.mode())
# print(bangladesh1.projection().crs().getInfo())
# print(bangladesh1.projection().transform())
print(bangladesh1.getInfo())

bangladesh2 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID2).first()).select(['flooded'])
bangladesh2 = bangladesh2.gt(0.5)
print(bangladesh2.getInfo())
# bangladesh2 = bangladesh2.resample(mode="bilinear")
# bangladesh2 = bangladesh2.reduceResolution(reducer=ee.Reducer.mode())

combo = ee.ImageCollection([bangladesh1, bangladesh2, filtered_occurrence, sea]).reduce(ee.Reducer.max())
# combo = ee.ImageCollection([bangladesh1, bangladesh2, filtered_occurrence]).reduce(ee.Reducer.max())
# print(type(combo))


# combo = combo.setDefaultProjection("EPSG:4326", [0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901]).reduceResolution(ee.Reducer.mode())
print(combo.getInfo())
# point = ee.Geometry.Point(89.97, 23.65)

# # Get the pixel values at the defined point
# pixel_values = combo.reduceRegion(
#     reducer=ee.Reducer.first(),
#     geometry=point,
#     scale=30
# )
# # Print the pixel values to the console
# print(pixel_values.getInfo())

# combo_gfd = ee.ImageCollection([bangladesh1, bangladesh2])
# filtered_occurrence = ee.ImageCollection([filtered_occurrence])

# combo = combo_gfd.merge(filtered_occurrence).max()

vis_params = {'min': 0, 'max': 1, 'palette': '#00FFFF'}

Map.centerObject(bangladesh1, 9)
Map.addLayer(combo.selfMask(), vis_params, 'combo - Inundation Extent')
Map

{'type': 'Image', 'bands': [{'id': 'flooded', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.002245788210298804, 0, 88.08430518433968, 0, -0.002245788210298804, 26.44864775268901]}]}
{'type': 'Image', 'bands': [{'id': 'flooded', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.002245788210298804, 0, 88.08430518433968, 0, -0.002245788210298804, 26.44864775268901]}]}
{'type': 'Image', 'bands': [{'id': 'flooded', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [7289, 4017], 'crs': 'EPSG:4326', 'crs_transform': [0.002245788210298804, 0, 73.38337555972372, 0, -0.002245788210298804, 31.47023019091714]}], 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[74.9189332257506, 31.47135703503598], [73.38186075192326, 31.471351758720346], [73.38206428482283, 22.447768263234146], [74.6631941194119, 22

Map(center=[26.980613506376038, 81.56867324442466], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
export_task = ee.batch.Export.image.toDrive(
    image=combo,
    description='BangladeshFloodedCombo_vF',
    folder='ProjectFlood',
    fileNamePrefix='BangladeshFloodedCombo_vF_128',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=[0.0905969391479492,0.0,88.08430518433968,0.0, -0.0905969391479492, 26.44864775268901],
    maxPixels=1e13  # Adjust maxPixels as needed
)
export_task.start()

# [0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901] approx 250m
# [0.0905969391479492,0.0,88.08430518433968,0.0,−0.0905969391479492,26.44864775268901] widthx128
# [0.0226492347869873, 0.0, 88.08430518433968, 0.0, -0.0226492347869873, 26.44864775268901] heightx256

In [ ]:
transform = [0.0226492347869873, 0.0, 88.08430518433968, 0.0, -0.0226492347869873, 26.44864775268901]
transform_high_res = [0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901]
gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
perm_water = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
inundation_images = []
bangladeshID1 = 1789
bangladeshID2 = 1791
bangladesh1 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID1).first()).select(['flooded']).toByte()
bangladesh2 = ee.Image(gfd.filterMetadata('id', 'equals', bangladeshID2).first()).select(['flooded']).toByte()

inundation_images.append(bangladesh1)
inundation_images.append(bangladesh2)

# Add permanent water overlay
water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', 12).first())
occurrence_band = water_overlay.select('monthly_recurrence')
water_overlay_filtered = occurrence_band.gt(0.5)
water_overlay_filtered = water_overlay_filtered.rename('flooded')
water_overlay_filtered = water_overlay_filtered.setDefaultProjection("EPSG:4326", transform_high_res)
inundation_images.append(water_overlay_filtered.toByte())

# Add sea overlay
sea = ee.Image(ee.ImageCollection('MODIS/006/MOD44W').filterDate('2015-01-01', '2015-01-02').first()).select(['water_mask'])
sea = sea.rename('flooded')
sea = sea.setDefaultProjection("EPSG:4326", transform_high_res)
inundation_images.append(sea.toByte())

# Export combo image to gdrive
combo = ee.ImageCollection(inundation_images).reduce(ee.Reducer.max())
print(combo.getInfo())

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=combo,
    description='BangladeshFloodedCombo_vF_high_res_shrink',
    folder='ProjectFlood',
    fileNamePrefix='BangladeshFloodedCombo_vF_high_res_shrink',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=transform,
    maxPixels=1e13  # Adjust maxPixels as needed
)

export_task.start()

In [6]:
export_task.status()

{'state': 'FAILED',
 'description': 'BangladeshFloodedCombo_vF',
 'priority': 100,
 'creation_timestamp_ms': 1723449382575,
 'update_timestamp_ms': 1723449392561,
 'start_timestamp_ms': 1723449385695,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'error_message': 'Too many input pixels per output pixel. Need 1628, but only 64 allowed.',
 'id': 'L62JYFJPA4YYSEB2AHLNIQDX',
 'name': 'projects/ee-varunkhandekar/operations/L62JYFJPA4YYSEB2AHLNIQDX'}

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=combo,
    description='BangladeshFloodedCombo_vF_highres',
    folder='ProjectFlood',
    fileNamePrefix='BangladeshFloodedCombo_vF_highres',
    region=bbox_ee,
    # crs='EPSG:4326',
    scale=30,  # Adjust the scale as needed
    # crsTransform=[0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901],
    maxPixels=1e13  # Adjust maxPixels as needed
)
export_task.start()

In [ ]:
export_task.status()

Rainfall Data

In [ ]:
# 2020365.21.nc
data = xr.open_dataset("2020365.21.nc", engine="netcdf4")
print(data)
# print(data.variables)



# Map = geemap.Map()
# Map.add_netcdf(
#     "2020365.21.nc",
#     variables=["v_wind"],
#     palette="coolwarm",
#     shift_lon=True,
#     layer_name="v_wind",
# )
# Map

In [ ]:
data = geemap.read_netcdf("2020365.21.nc")

# precipitation_at_time = data['precipitation'].isel(time=0)
data

In [ ]:
# data.sizes
# data_at_time = data.isel(time=0).squeeze()
# data_at_time.dims

In [ ]:
# Extract precipitation data
data = xr.open_dataset("2020365.21.nc", engine="netcdf4")
precipitation = data['precipitation']
precipitation.plot()
print(precipitation.dtype)
print(precipitation.min())

In [ ]:

# Set the spatial dimensions
precipitation.rio.set_spatial_dims('lon', 'lat', inplace=True)
precipitation.rio.write_crs("EPSG:4326", inplace=True)

# Save the precipitation data to GeoTIFF
precipitation.rio.to_raster("precipitation_data.tif")

# Initialize the map
Map = geemap.Map()

# Add the GeoTIFF file to the map
geotiff_path = "precipitation_data.tif"
Map.add_raster(geotiff_path, vmin=0, vmax = 3, layer_name="Precipitation")

# image = geemap.ee_image_from_drive(geotiff_path)

# # Define visualization parameters
# vis_params = {
#     'min': 0,
#     'max': 10,  # Adjust these values based on your data range
#     'palette': ['blue', 'green', 'red']
# }

# # Add the image to the map
# Map.addLayer(image, vis_params, 'Precipitation')

# Display the map
fc = ee.FeatureCollection(ee_geometry)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')
Map

In [ ]:
# from rasterio.enums import Resampling
# Open the NetCDF file
data = xr.open_dataset("2020365.21.nc", engine="netcdf4")
precipitation = data['precipitation']
precipitation.max()

In [ ]:

# Set spatial dimensions and CRS
precipitation.rio.set_spatial_dims('lon', 'lat', inplace=True)
precipitation.rio.write_crs("EPSG:4326", inplace=True)

# Define the output path for the GeoTIFF file
output_path = 'precipitation_data_Bangladesh_new.tif'


# Clip the precipitation data using the polygon
precipitation_clipped = precipitation.rio.clip(geojson, all_touched=True)
precipitation_clipped.max()

In [ ]:

# Save the clipped data to GeoTIFF
precipitation_clipped.rio.to_raster(output_path)

print(f"Clipped image saved to {output_path}")

# Map = geemap.Map()

# Add the GeoTIFF file to the map
# geotiff_path = "precipitation_data_Bangladesh_new.tif"
# Map.add_raster(geotiff_path, vmin=0, vmax = 3, layer_name="Precipitation")

# # Display the map
# fc = ee.FeatureCollection(ee_geometry)
# style = {'color': 'ffff00ff', 'fillColor': '00000000'}
# Map.addLayer(fc.style(**style), {}, 'ROI')
# Map

In [ ]:
with rasterio.open('precipitation_data_Bangladesh_new.tif') as src:
    
    src_transform = src.transform
    data = src.read()
    print(data.dtype)

Soil Moisture

In [ ]:
c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'format': 'zip',
        'year': '2019',
        'day':'01',
        'variable': 'surface_soil_moisture',
        'type_of_sensor': 'active',
        'time_aggregation': 'day_average',
        'month': '01',
        'type_of_record': 'cdr',
        'version': 'v202212',
    },
    'download.zip'
)
with zipfile.ZipFile('download.zip', 'r') as zip_ref:
    file_list = zip_ref.namelist()
    zip_ref.extract(file_list[0], path='.')

    new_path = os.path.join('.', f'soil_moisture_bangladesh_test.nc')
    os.rename(file_list[0], new_path)
    print(f'File extracted and renamed to: {new_path}')
os.remove('download.zip')



In [ ]:
# from rasterio.enums import Resampling
# Open the NetCDF file
data = xr.open_dataset("C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-DAILY-20190104000000-TCDR-v202212.0.0.nc", engine="netcdf4")
data

In [ ]:

soil_moisture = data['sm']

# Set spatial dimensions and CRS
soil_moisture.rio.set_spatial_dims('lon', 'lat', inplace=True)
soil_moisture.rio.write_crs("EPSG:4326", inplace=True)
soil_moisture.max()


In [ ]:

soil_moisture.rio.to_raster("soil_moisture_test.tif")

# Initialize the map
Map = geemap.Map()

# Add the GeoTIFF file to the map
geotiff_path = "soil_moisture_test.tif"
Map.add_raster(geotiff_path, vmin=0, vmax = 100, layer_name="Soil_Moisture")
Map

# Display the map
# fc = ee.FeatureCollection(ee_geometry)
# style = {'color': 'ffff00ff', 'fillColor': '00000000'}
# Map.addLayer(fc.style(**style), {}, 'ROI')
# Map


In [ ]:
# Define the output path for the GeoTIFF file
output_path = 'soil_moisture_test_Bangladesh.tif'


# Get the bounding box
minx, miny, maxx, maxy = polygon.bounds

print(f"Bounding box: ({minx}, {miny}, {maxx}, {maxy})")


In [ ]:

# Convert the polygon to GeoJSON format
geojson = [mapping(polygon)]

# Clip the precipitation data using the polygon
soil_moisture_clipped = soil_moisture.rio.clip(geojson, all_touched=True)

# Save the clipped data to GeoTIFF
soil_moisture_clipped.rio.to_raster(output_path)

print(f"Clipped image saved to {output_path}")

Map = geemap.Map()

# Add the GeoTIFF file to the map
geotiff_path = "soil_moisture_test.tif"
Map.add_raster(geotiff_path, vmin=0, vmax = 100, layer_name="Soil_Moisture")

# Display the map
fc = ee.FeatureCollection(ee_geometry)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')
Map

In [ ]:
c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'format': 'zip',
        'year': '2019',
        'day':'01',
        'variable': 'surface_soil_moisture',
        'type_of_sensor': 'active',
        'time_aggregation': 'day_average',
        'month': '01',
        'type_of_record': 'cdr',
        'version': 'v202212',
    },
    'download.zip'
)
new_path = os.path.join('.', f'soil_moisture_bangladesh_test.nc')
with zipfile.ZipFile('download.zip', 'r') as zip_ref:
    file_list = zip_ref.namelist()
    zip_ref.extract(file_list[0], path='.')

    
    os.rename(file_list[0], new_path)
    print(f'File extracted and renamed to: {new_path}')
os.remove('download.zip')


In [ ]:
new_path = os.path.join('.', f'soil_moisture_bangladesh_test.nc')
geojson = [mapping(polygon)] # Use GeoJSON of the shape
with xr.open_dataset(new_path, engine="netcdf4") as data:
    data = data['sm']
    data.rio.set_spatial_dims('lon', 'lat', inplace=True)
    data.rio.write_crs("EPSG:4326", inplace=True)
    
    data_clipped = data.rio.clip(geojson, all_touched=True)
data_clipped.to_netcdf(new_path)
# data_clipped.rio.to_raster('soil_moisture_bangladesh_test_clipping.tif')



In [ ]:
print(data_clipped.to_dataframe().values)

In [ ]:
import numpy as np

df = data_clipped.to_dataframe()
nan_indices = np.isnan(df['sm'])
nan_indices = list(df.index[nan_indices].droplevel('time'))
# nan_indices.droplevel('time')

nan_indices

In [ ]:
pivot_df = df.reset_index().pivot(index='lat', columns='lon', values='sm')
pivot_df = pivot_df.iloc[::-1]
pivot_df


In [ ]:

plt.figure(figsize=(10, 8))
# plt.imshow(pivot_df, cmap='viridis', origin='lower', extent=[-180, 180, -90, 90])
lat_min, lat_max = pivot_df.index.min(), pivot_df.index.max()
lon_min, lon_max = pivot_df.columns.min(), pivot_df.columns.max()
plt.imshow(pivot_df, cmap='viridis', origin='upper', extent = [lon_min, lon_max, lat_min, lat_max])
plt.colorbar(label='Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Grid Plot of Values')
plt.show()

In [ ]:
def get_band_number_by_name(src, band_name):
    # Assuming the band names are stored in the metadata under 'band_descriptions'
    descriptions = src.descriptions
    for i, desc in enumerate(descriptions):
        if desc == band_name:
            return i + 1  # Bands are 1-indexed in rasterio
    raise ValueError(f"Band name '{band_name}' not found")

def element_wise_average(arr1, arr2):
    # Create masks for NaN values
    mask1 = np.isnan(arr1)
    mask2 = np.isnan(arr2)
    
    # Replace NaNs in arr1 with values from arr2
    arr1[mask1] = arr2[mask1]
    
    # Replace NaNs in arr2 with values from arr1
    arr2[mask2] = arr1[mask2]
    
    # Calculate the average
    avg_array = (arr1 + arr2) / 2
    
    return avg_array

In [ ]:
# SMAP L3
dataset = ee.ImageCollection('NASA/SMAP/SPL3SMP_E/005')
im = ee.Image(dataset.filterDate('2019-01-01', '2019-01-02').first())
print(im.getInfo())

# ee_geometry.getInfo()['coordinates']
print(type(im))


In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=im,
    description='SMAP_test_L3',
    folder='ProjectFlood',
    fileNamePrefix='SMAP_test_L3',
    region=ee_geometry.getInfo()['coordinates'],
    scale=11000,  # Adjust the scale as needed
    maxPixels=1e13  # Adjust maxPixels as needed
)
export_task.start()


In [ ]:
export_task.status()

In [ ]:
with rasterio.open("SMAP_test_L3.tif") as src:
    band_name = "soil_moisture_am"
    band_number = get_band_number_by_name(src, band_name)
    # print(band_number)
    band1 = src.read(band_number)  # Read the first band
    print(band1)

    band_name = "soil_moisture_pm"
    band_number = get_band_number_by_name(src, band_name)
    # print(band_number)
    band2 = src.read(band_number)  # Read the first band
    print("=================")
    print(band2)


    avg_band = element_wise_average(band1.copy(), band2.copy())
    print("=================")
    print(avg_band)
    
    # Get the coordinates of the pixels
    rows, cols = np.indices(avg_band.shape)
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lats = np.array(ys).flatten()
    lons = np.array(xs).flatten()
    values = avg_band.flatten()

    # Create a DataFrame
    df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'value': values
    })

df = df.pivot_table(index='latitude', columns='longitude', values='value')
df = df.iloc[::-1]

# df = df/0.9
# print(df.head())


In [ ]:
plt.figure(figsize=(10, 8))
# plt.imshow(pivot_df, cmap='viridis', origin='lower', extent=[-180, 180, -90, 90])
lat_min, lat_max = df.index.min(), df.index.max()
lon_min, lon_max = df.columns.min(), df.columns.max()
plt.imshow(df, cmap='viridis', origin='upper', extent = [lon_min, lon_max, lat_min, lat_max])
plt.colorbar(label='Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Grid Plot of Values')
plt.show()

In [ ]:
with rasterio.open("SMAP_test_L4.tif") as src:
    band_name = "sm_surface"
    band_number = get_band_number_by_name(src, band_name)
    # print(band_number)
    band = src.read(band_number)  # Read the first band
    # print(type(band))
    # Get the coordinates of the pixels
    rows, cols = np.indices(band.shape)
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lats = np.array(ys).flatten()
    lons = np.array(xs).flatten()
    values = band.flatten()

    # Create a DataFrame
    df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'value': values
    })

df = df.pivot_table(index='latitude', columns='longitude', values='value')
df = df.iloc[::-1]
df = df/0.9
# df


In [ ]:
plt.figure(figsize=(10, 8))
# plt.imshow(pivot_df, cmap='viridis', origin='lower', extent=[-180, 180, -90, 90])
lat_min, lat_max = df.index.min(), df.index.max()
lon_min, lon_max = df.columns.min(), df.columns.max()
plt.imshow(df, cmap='viridis', origin='upper', extent = [lon_min, lon_max, lat_min, lat_max])
plt.colorbar(label='Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Grid Plot of Values')
plt.show()

In [9]:
# Load the dataset and filter for the month of December
dataset = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
im = ee.Image(dataset.filterDate('2019-01-01', '2019-01-02').first())

# im = im.toFloat()
im = im.select("volumetric_soil_water_layer_1")
print(im.getInfo())
# ee_geometry.getInfo()['coordinates']
print(type(im))
Map = geemap.Map()

vis_params1 = {'min': 0, 'max': 1, 'palette': ['#000000', '#FFFFFF']}
Map.addLayer(im, vis_params1, 'Landsat')
Map

{'type': 'Image', 'bands': [{'id': 'volumetric_soil_water_layer_1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [3601, 1801], 'crs': 'EPSG:4326', 'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}], 'version': 1724130942698911, 'id': 'ECMWF/ERA5_LAND/DAILY_AGGR/20190101', 'properties': {'system:time_start': 1546300800000, 'month': 1, 'year': 2019, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]}, 'system:time_end': 1546387200000, 'system:asset_size': 940588144, 'day': 1, 'system:index': '20190101'}}
<class 'ee.image.Image'>


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
perm_water = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', 1).first())
occurrence_band = water_overlay.select('monthly_recurrence')
water_overlay_filtered = occurrence_band.gt(0.99) 


# Mask the soil moisture image using the sea mask
masked_soil_moisture = im.updateMask(water_overlay_filtered.eq(1))

# Reduce the masked soil moisture image to get the average soil moisture value
average_soil_moisture = masked_soil_moisture.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=bbox_ee,
    crs='EPSG:4326',
    crsTransform=[0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901]
).getInfo()

im2 = im.unmask(average_soil_moisture['volumetric_soil_water_layer_1'])
# # Extract the mean value from the result
print(average_soil_moisture['volumetric_soil_water_layer_1'])

Map = geemap.Map()

vis_params1 = {'min': 0, 'max': 1, 'palette': ['#000000', '#FFFFFF']}
Map.addLayer(masked_soil_moisture, vis_params1, 'Landsat')
Map

0.20474687132083308


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:

# Replace N/A values in the original soil moisture array with the calculated mean value
na_mask = np.isnan(soil_moisture_array)
soil_moisture_array[na_mask] = mean_value

In [ ]:
im = im.unmask(1)
# im = im.resample(mode="bicubic")
export_task = ee.batch.Export.image.toDrive(
    image=im,
    description='ERA5-LAND',
    folder='ProjectFlood',
    fileNamePrefix='ERA5-LAND',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=[0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901],
    maxPixels=1e8  # Adjust maxPixels as needed
)
export_task.start()


In [ ]:
export_task.status()

In [ ]:
with rasterio.open('ERA5-LAND.tif') as src:
    # Read the existing data
    data_era5 = src.read()
    
data_era5.nbytes

In [ ]:
with rasterio.open('ERA5-LAND_bilinear.tif') as src:
    # Read the existing data
    data_era5_bl = src.read()
    
data_era5_bl.nbytes

In [ ]:
with rasterio.open("ERA5-LAND.tif") as src:
    band_name = "volumetric_soil_water_layer_1"
    band_number = get_band_number_by_name(src, band_name)
    print(band_number)
    band = src.read(band_number)
    # Get the coordinates of the pixels
    rows, cols = np.indices(band.shape)
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lats = np.array(ys).flatten()
    lons = np.array(xs).flatten()
    values = band.flatten()

    # Create a DataFrame
    df = pd.DataFrame({
        'latitude': lats,
        'longitude': lons,
        'value': values
    })

df = df.pivot_table(index='latitude', columns='longitude', values='value')
df = df.iloc[::-1]
# df = df/0.9
df


In [ ]:
plt.figure(figsize=(10, 8))
# plt.imshow(pivot_df, cmap='viridis', origin='lower', extent=[-180, 180, -90, 90])
lat_min, lat_max = df.index.min(), df.index.max()
lon_min, lon_max = df.columns.min(), df.columns.max()
plt.imshow(df, cmap='viridis', origin='upper', extent = [lon_min, lon_max, lat_min, lat_max])
plt.colorbar(label='Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Grid Plot of Values')
plt.show()

In [ ]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis',
        'variable': 'volumetric_soil_water_layer_1',
        'year': '2019',
        'month': '01',
        'time': '00:00',
        'area': [
            26.4, 88.12, 20.63,
            92.63,
        ],
        'format': 'grib',
    },
    'era5_land_soil_moisture_test.grib')

In [ ]:

ds = cfgrib.open_dataset('era5_land_soil_moisture_test.grib')

In [ ]:
# Open the NetCDF file
nc_file = 'input.nc'
variable_name = 'surface_soil_moisture'
dataset = gdal.Open(f'NETCDF:"{new_path}":{variable_name}')

# Check if the dataset was successfully opened
if dataset is None:
    print("Failed to open the NetCDF file.")
else:
    # Define the output GeoTIFF file
    tif_file = os.path.basename(new_path).split('.')[0] + '.tif'
    
    # Convert the dataset to GeoTIFF format
    gdal.Translate(tif_file, dataset, format='GTiff', noData=-9999)

    print("Conversion complete. Output saved to", tif_file)

In [ ]:
os.path.basename(new_path).split('.')[0] + '.tif'

In [ ]:
Map = geemap.Map()

# Add the GeoTIFF file to the map
geotiff_path = 'soil_moisture_bangladesh_test_clipping.tif'
Map.add_raster(geotiff_path, vmin=0, vmax = 100, layer_name="Soil_Moisture")
Map

Testing sending files from local to gdrive

In [ ]:

def authenticate(config_file_path="config.json"):
    """Authenticate using PyDrive and return a GoogleDrive object."""
    with open(config_file_path) as config_file:
        config = json.load(config_file)
    google_drive_credentials_path = config['google_drive_credentials']
    google_drive_oauth_path = config['google_drive_oauth']

    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(google_drive_credentials_path)

    # Try to load saved credentials
    gauth.LoadCredentialsFile(google_drive_oauth_path)
    if gauth.credentials is None:
        try:
            gauth.LocalWebserverAuth()  # Creates local webserver and auto handles authentication
        except:
            gauth.CommandLineAuth()  # Use this if LocalWebserverAuth fails
        gauth.SaveCredentialsFile(google_drive_oauth_path)
    elif gauth.access_token_expired:
        gauth.Refresh()
        gauth.SaveCredentialsFile(google_drive_oauth_path)
    else:
        gauth.Authorize()

    return GoogleDrive(gauth)

In [ ]:
with open("config.json") as config_file:
    config = json.load(config_file)
google_drive_credentials_path = config['google_drive_credentials']
google_drive_oauth_path = config['google_drive_oauth']

gauth = GoogleAuth()


In [ ]:
gauth.LocalWebserverAuth()

In [ ]:

gauth.LoadClientConfigFile(google_drive_credentials_path)


In [ ]:

# Try to load saved credentials
gauth.LoadCredentialsFile(google_drive_oauth_path)
if gauth.credentials is None:
    try:
        gauth.LocalWebserverAuth()  # Creates local webserver and auto handles authentication
    except:
        gauth.CommandLineAuth()  # Use this if LocalWebserverAuth fails
    gauth.SaveCredentialsFile(google_drive_oauth_path)
elif gauth.access_token_expired:
    gauth.Refresh()
    gauth.SaveCredentialsFile(google_drive_oauth_path)
else:
    gauth.Authorize()

In [ ]:
gauth.LoadCredentialsFile(google_drive_oauth_path)
gauth.LocalWebserverAuth()

In [ ]:

# Get folder id from config
with open('config.json') as config_file:
    config = json.load(config_file)
folder_id = config['soil_moisture_folder_id']

# drive = GoogleDrive(gauth)
drive = authenticate('config.json')


file_name = os.path.basename("./download.nc")
overwrite=True
if overwrite:
    # Search for the file with the same name in the target folder; delete if it exists
    file_list = drive.ListFile({'q': f"title = '{file_name}' and '{folder_id}' in parents and trashed=false"}).GetList()
    for file in file_list:
        file.Delete()

file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}]})
file.SetContentFile("./download.nc")
file.Upload()
file = None

Testing moving files from shared to personal GDrive

In [ ]:
"""List files shared with the user."""
file_list = drive.ListFile({'q': "'me' in owners"}).GetList()
file_list

In [ ]:
timestamp = pd.Timestamp('2024-06-26')

start_time = timestamp - pd.Timedelta(days=2)
end_time = timestamp + pd.Timedelta(days=1)

timestamps = pd.date_range(start=start_time, end=end_time, freq='3h')
timestamps = timestamps.tolist()[:-1]


print(f"{timestamps[3].hour:02}", timestamps[0], timestamps[0].year)

In [ ]:
def generate_timestamps(date, days_before: int, days_after: int):
    start_time = date - pd.Timedelta(days=days_before)
    end_time = date + pd.Timedelta(days=days_after)

    timestamps = pd.date_range(start=start_time, end=end_time, freq='3h')
    # don't include final forecast as it takes us into day t+days_after+1
    return timestamps.tolist()[:-1]

def convert_date_to_MSWEP_file_name(timestamp):
    return f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.nc"

def generate_files_of_interest(timestamps):
    return [convert_date_to_MSWEP_file_name(timestamp) for timestamp in timestamps]

timestamp = pd.Timestamp('2014-06-26')
timestamps = generate_timestamps(timestamp, 2, 1)
file_names = generate_files_of_interest(timestamps)

with open("config.json") as config_file:
    config = json.load(config_file)
folder_id = config['MSWEP_Past_3hr_folder_id']

# query = f"title='{file_names[0]}' and '{folder_id}' in parents"
# query


In [ ]:
files = []
for file_name in file_names:
    query = f"title='{file_name}' and '{folder_id}' in parents"
    file_list = drive.ListFile({'q': query}).GetList()
    files.extend(file_list)

In [ ]:
print(type(files[0]))
# files[0].GetContentFile(file_names[0])
files[0]['id']
# files[0].keys()

In [ ]:
new_file = drive.CreateFile({'id': files[0]['id']})
new_path = os.path.join("./data/temp", f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.nc")
temp_path = os.path.join("./data/temp", f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_tmp.nc")
new_file.GetContentFile(new_path)

data = xr.open_dataset(new_path, engine="netcdf4")
data

In [ ]:
geojson = [mapping(polygon)] # Use GeoJSON of the shape
with xr.open_dataset(new_path, engine="netcdf4") as data:
    data = data['precipitation']
    data.rio.set_spatial_dims('lon', 'lat', inplace=True)
    data.rio.write_crs("EPSG:4326", inplace=True)
    
    data_clipped = data.rio.clip(geojson, all_touched=True)

data_clipped.to_netcdf(temp_path, mode='w')
os.remove(new_path)
os.rename(temp_path, new_path)

In [ ]:
data_clipped.to_netcdf(new_path, mode='w')

In [ ]:
with open("config.json") as config_file:
    config = json.load(config_file)
target_folder_id = config['bangladesh_rainfall_folder_id']

file = files[0]
file.FetchMetadata()  # Get the file metadata
copied_file = drive.CreateFile({
    'title': file['title'],
    'parents': [{'id': target_folder_id}],
    'mimeType': file['mimeType']
})

# Copy the content
copied_file.SetContentString(file.GetContentString())
copied_file.Upload()

# copied_file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}]})
# copied_file.SetContentFile(file)
# copied_file.Upload()

GDAL testing

In [ ]:
# 2020365.21.nc

# Path to your NetCDF file
netcdf_file = '2020365.21.nc'
output_tif_file = 'gdal_testing.tif'
dataset = gdal.Open('NETCDF:"{}":precipitation'.format(netcdf_file))
# dataset

driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.CreateCopy(output_tif_file, dataset, 0)

output_dataset.SetProjection('EPSG:4326')

dataset = None
output_dataset = None

print(f"Converted {netcdf_file} to {output_tif_file} with EPSG:4326")

Upsampling

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

# File paths
file_path_1 = 'BangladeshFloodedCombo.tif'
# BangladeshFloodedCombo
# ERA5-LAND_withmask
file_path_2 = 'precipitation_data_Bangladesh.tif'
output_path = 'upsampled_precipitation_data_Bangladesh.tif'

# Open the first image to get its resolution
image_1 = rasterio.open(file_path_1)
resolution_1 = image_1.res
print(resolution_1)

# Open the second image to resample
image_2 = rasterio.open(file_path_2)
resolution_2 = image_2.res
print(resolution_2)
print(image_1.crs)
print(image_2.crs)
print(image_1.width)
print(image_1.height)
print(*image_1.bounds)
print(image_1.transform)
print(image_1.meta)
print(image_2.meta)


In [ ]:
with rasterio.open('precipitation_data_Bangladesh_new.tif') as src:
    
    src_transform = src.transform
    data = src.read()
    print(data.shape)
    with rasterio.open('BangladeshFloodedCombo_MASTER.tif') as target:
        target_transform = target.transform
        target_width = target.width
        target_height = target.height
        kwargs = src.meta
        kwargs.update({
            'height': target_height,
            'width': target_width,
            'transform': target_transform
        })
    # print(kwargs)
    with rasterio.open('precipitation_upsampled_no_compress.tif', 'w', **kwargs) as dst:

        for i, band in enumerate(data, 1):
            dest = np.zeros((target_height, target_width), dtype=np.float32)

            reproject(
                source=band,
                destination=dest,
                src_transform=src_transform,
                src_crs=src.crs,
                dst_transform=target_transform,
                dst_crs=src.crs,
                resampling=Resampling.bilinear)
            print(dest.shape)
            print(dest.max())


            dst.write(dest, indexes=i)
    
print(data.max())


In [ ]:

with rasterio.open('2007174.06.tif', 'r') as src:
    data = src.read(1)  # Assuming it is a single band raster
    data = data * 10000
    print(data.max())
    data = data.astype(np.uint16)
    
    
    new_meta = src.meta.copy()
    new_meta.update({
        'dtype': 'uint16',
        'nodata': 0,
        'compress': 'lzw'
    })

    with rasterio.open('2007174.06_no_compress.tif', 'w', **new_meta) as dst:
        dst.write(data, 1)

# Verify the dtype of the new file
with rasterio.open('2007174.06_no_compress.tif') as new_src:
    print(new_src.meta)
    new_data = new_src.read()

print(new_data.max())

In [ ]:
with rasterio.open('precipitation_upsampled_no_compress.tif', 'r+') as src:
    # Read the existing data
    data = src.read()
    
data.min()

In [ ]:
with rasterio.open('precipitation_upsampled.tif') as data:
    dataset = data.read()

print(np.isnan(dataset).any())

In [ ]:
import numpy as np
from rasterio.warp import reproject, Resampling

# File paths
source_path = 'precipitation_data_Bangladesh_new.tif'
target_path = 'precipitation_upsampled.tif'

# Open source file
with rasterio.open(source_path) as src:
    src_transform = src.transform
    data = src.read()
    src_crs = src.crs
    src_meta = src.meta
    
    # Open target file to get target dimensions and transform
    with rasterio.open('BangladeshFloodedCombo_MASTER.tif') as target:
        target_transform = target.transform
        target_width = target.width
        target_height = target.height
        target_crs = target.crs
        
        # Update metadata for the new file
        kwargs = src.meta
        kwargs.update({
            'height': target_height,
            'width': target_width,
            'transform': target_transform,
            'crs': target_crs,
            'compress': 'lzw'
        })
        print(kwargs)

    
    with rasterio.open(target_path, 'w', **kwargs) as dst:
        for i, band in enumerate(data, 1): # Go through bands
            dest = np.zeros((target_height, target_width), dtype=np.float32)
            
            reproject(
                source=band,
                destination=dest,
                src_transform=src_transform,
                src_crs=src_crs,
                dst_transform=target_transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )
            print(dest)
            # data[i-1] = dest
            dst.write(dest, indexes=i)



In [ ]:
# File paths
file_path_1 = 'BangladeshFloodedCombo_MASTER.tif'

# Open the first image to get its resolution
with rasterio.open(file_path_1) as image_1:
    resolution_1 = image_1.res
    profile_1 = image_1.profile


print(profile_1)
print(type(profile_1['transform']))
print("a:", profile_1['transform'].a)
print("b:", profile_1['transform'].b)
print("c:", profile_1['transform'].c)
print("d:", profile_1['transform'].d)
print("tx (e):", profile_1['transform'].e)
print("ty (f):", profile_1['transform'].f)


In [ ]:
from rasterio.enums import Resampling

# File paths
input_file_path = 'ERA5-LAND_withmask.tif'
output_file_path = 'ERA5-LAND_withmask_readable.tif'

# Open the input image
with rasterio.open(input_file_path) as src:
    # Read the image data
    data = src.read()

    # Normalize the data for 8-bit conversion
    data_min = data.min()
    data_max = data.max()
    data = (data - data_min) / (data_max - data_min) * 255
    data = data.astype(np.float16)
    
    # Define a new profile with updated dtype and count
    profile = src.profile
    profile.update(dtype=rasterio.uint8, count=data.shape[0])

    # Save the new image as a readable TIFF
    with rasterio.open(output_file_path, 'w', **profile) as dst:
        dst.write(data)

print(f"Readable TIFF image saved as {output_file_path}")


Affine transformations

In [ ]:
# Open the second GeoTIFF file
file_path_2 = 'BangladeshFloodedCombo.tif'
# upsampled_precipitation_data_Bangladesh_scipy_new
# ERA5-LAND_withmask_readable
# BangladeshFloodedCombo
dataset_2 = rasterio.open(file_path_2)

# Extract the metadata
metadata_2 = dataset_2.meta
band_count_2 = dataset_2.count
bounds_2 = dataset_2.bounds
crs_2 = dataset_2.crs
transform_2 = dataset_2.transform

metadata_summary_2 = {
    "Driver": metadata_2['driver'],
    "Width": metadata_2['width'],
    "Height": metadata_2['height'],
    "Count of Bands": band_count_2,
    "Dtype": metadata_2['dtype'],
    "Bounds": bounds_2,
    "Coordinate Reference System": crs_2,
    "Transform": transform_2
}

metadata_summary_2


In [ ]:
a = pd.read_csv("to_rerun.csv", header=None)
a = list(a.values.flatten())
a = [pd.Timestamp(i) for i in a]
a

In [ ]:
def convert_date_to_MSWEP_file_name(timestamp: pd.Timestamp):
    return f"{timestamp.year}{timestamp.dayofyear:03}.{timestamp.hour:02}.nc"

convert_date_to_MSWEP_file_name(a[0])

In [ ]:
gauth = GoogleAuth()
with open("config.json") as config_file:
    config = json.load(config_file)
google_drive_credentials_path = config['google_drive_credentials']
google_drive_oauth_path = config['google_drive_oauth']
gauth.LoadClientConfigFile(google_drive_credentials_path)
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

In [ ]:
query = f"title='2017085.00.nc' and '1DVR90Ud1C444bTOPeENX-3I7tgqPLnao' in parents"
file_list = drive.ListFile({'q': query}).GetList()
file_list

In [ ]:
def store_flood_dates(events_dict, config_file):
    dates = [i for i in events_dict.keys()]
    df = pd.DataFrame(dates, columns=["Dates"])
    with open(config_file) as config_file:
        config = json.load(config_file)
    target_path = config["final_flood_dates"]
    df.to_excel(target_path, index=False)

In [ ]:
shape = bbox
zzz = "/home/vkhandekar/project_flood/archive/2014175.00.nc"
yyy = "/home/vkhandekar/project_flood/archive/2014175_testing_without_GDAL_20240731.tif"
geojson = [mapping(shape)] # Use GeoJSON of the shape
with xr.open_dataset(zzz, engine="netcdf4") as data:
    data = data['precipitation']
    data.rio.set_spatial_dims('lon', 'lat', inplace=True)
    data.rio.write_crs("EPSG:4326", inplace=True)
    
    data_clipped = data.rio.clip(geojson, all_touched=True)
    # data_clipped = ((data_clipped*1000)//1).astype(np.uint16) # QUANTIZATION
    data_clipped = data_clipped.squeeze()
print(type(data_clipped.dtype))
print(data_clipped.values.mean())

In [ ]:
def get_transform_from_xarray(data_array):
    # Assuming the coordinates are named 'lon' and 'lat' and are regularly spaced
    lon_res = data_array.lon[1] - data_array.lon[0]  # Longitude resolution
    lat_res = data_array.lat[1] - data_array.lat[0]  # Latitude resolution
    transform = from_origin(data_array.lon.min(), data_array.lat.max(), lon_res, abs(lat_res))
    return transform

In [ ]:
metadata = {
        'driver': 'GTiff',
        'height': data_clipped.shape[0],
        'width': data_clipped.shape[1],
        'count': 1,
        'dtype': data_clipped.dtype,  # Using the numpy dtype directly
        'crs': 'EPSG:4326',  # Assuming the CRS is WGS84
        'transform': get_transform_from_xarray(data_clipped)
    }
with rasterio.open(yyy, 'w', **metadata) as dst:
    dst.write(data_clipped.values, 1)

In [ ]:
print(os.getcwd())

with open("/home/vkhandekar/project_flood/static/local_data_paths.json") as config_file:
    config = json.load(config_file)
os.listdir(config["topology_path_2044_2573"])
